# Data Cleaning
[www.vexpower.com](www.vexpower.com)

In [1]:
# Set the right folder

import sys
import os
import pandas as pd
import numpy as np

if not os.path.isdir("mmm"):
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

import mmm

### Facebook Ads
Load and clean Facebook Ads data

In [2]:
# Load example Facebook Campaign data
file_name = "Goolybib-Campaigns-Mar-19-2021-Apr-17-2021.csv"
data = pd.read_csv('../data/'+file_name)
data.head()

,Reporting Starts,Reporting Ends,Campaign Name,Campaign Delivery,Attribution Setting,Results,Result Indicator,Reach,Frequency,Cost per Results,...,Impressions,"CPM (Cost per 1,000 Impressions) (USD)",Link Clicks,CPC (Cost per Link Click) (USD),CTR (Link Click-Through Rate),Clicks (All),CTR (All),CPC (All) (USD),Website Purchases,On-Facebook Purchases
0,2021-04-17,2021-04-17,Prospecting - Broad - US,active,7-day click or 1-day view,3,actions:offsite_conversion.fb_pixel_purchase,20103,1.323783,165.582500,...,26612,24.888396,148,4.475203,0.556140,218,0.819179,3.038211,4,NaN
1,2021-04-17,2021-04-17,Prospecting - Broad - Rest of World,active,7-day click or 1-day view,5,actions:offsite_conversion.fb_pixel_purchase,27700,1.232058,83.486000,...,34128,12.231306,150,2.782867,0.439522,263,0.770628,1.587186,2,NaN
2,2021-04-17,2021-04-17,Remarketing - RoW,active,7-day click or 1-day view,8,actions:offsite_conversion.fb_pixel_purchase,13378,1.605322,60.132222,...,21476,25.199758,167,3.240659,0.777612,280,1.303781,1.932821,3,NaN
3,2021-04-17,2021-04-17,Remarketing US,active,7-day click or 1-day view,15,actions:offsite_conversion.fb_pixel_purchase,16205,2.217834,80.990000,...,35940,36.055648,154,8.414545,0.428492,257,0.715081,5.042179,5,NaN
4,2021-04-16,2021-04-16,Prospecting - Broad - US,active,7-day click or 1-day view,3,actions:offsite_conversion.fb_pixel_purchase,19460,1.206937,189.393333,...,23487,24.191255,168,3.382024,0.715289,272,1.158087,2.088897,3,NaN


In [3]:
from mmm.select import get_cols_containing
from mmm.clean import rename_column

spend_col = get_cols_containing(data, 'Amount Spent')[0] # needed because column name contains currency

df = data.copy()
rename_column(df, 'Reporting Starts', 'date')
rename_column(df, spend_col, 'facebook_cost')
rename_column(df, 'Campaign Name', 'campaign_name')

df = df[['date', 'facebook_cost', 'campaign_name']]
df.head()

,date,facebook_cost,campaign_name
0,2021-04-17,662.33,Prospecting - Broad - US
1,2021-04-17,417.43,Prospecting - Broad - Rest of World
2,2021-04-17,541.19,Remarketing - RoW
3,2021-04-17,1295.84,Remarketing US
4,2021-04-16,568.18,Prospecting - Broad - US


In [4]:
from mmm.clean import unstack_data

df = unstack_data(df, metric_column='facebook_cost', unstack_column='campaign_name')

df.head()

,date,Prospecting - Broad - Rest of World | facebook_cost,Prospecting - Broad - US | facebook_cost,Prospecting - Lookalike - US | facebook_cost,Remarketing - RoW | facebook_cost,Remarketing US | facebook_cost
0,2021-03-19,0.00,0.00,827.65,407.48,1445.37
1,2021-03-20,0.00,0.00,0.00,470.21,1526.23
2,2021-03-21,0.00,0.00,813.31,393.15,1413.45
3,2021-03-22,243.30,271.67,746.86,480.06,1437.29
4,2021-03-23,296.28,288.62,0.00,496.98,1443.81


In [6]:
df.to_csv('../data/facebook_clean.csv', index=False)

### Google Ads
Load and clean Google Ads data

In [7]:
# Load example Google Campaign data
file_name = "Day (1).csv"
data = pd.read_csv('../data/'+file_name)
data.head()

,Day,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"March 24, 2021 - April 22, 2021",NaN,NaN,NaN,NaN
1,Day,Campaign,Currency,Impressions,Cost
2,2021-03-24,US // Growth Hacking,USD,2109,72.66
3,2021-03-24,EU // Growth Hacking,USD,29,4.08
4,2021-03-24,EU // Agency Types,USD,177,9.82


In [8]:
data = pd.read_csv('../data/'+file_name, skiprows=2)
data.head()

,Day,Campaign,Currency,Impressions,Cost
0,2021-03-24,US // Growth Hacking,USD,2109,72.66
1,2021-03-24,EU // Growth Hacking,USD,29,4.08
2,2021-03-24,EU // Agency Types,USD,177,9.82
3,2021-03-24,UK-only // Growth Hacking,USD,236,4.22
4,2021-03-25,US // Growth Hacking,USD,1379,44.98


In [9]:
from mmm.clean import rename_column

df = data.copy()
rename_column(df, 'Day', 'date')
rename_column(df, 'Cost', 'google_cost')
rename_column(df, 'Campaign', 'campaign_name')

df = df[['date', 'google_cost', 'campaign_name']]
df.head()

,date,google_cost,campaign_name
0,2021-03-24,72.66,US // Growth Hacking
1,2021-03-24,4.08,EU // Growth Hacking
2,2021-03-24,9.82,EU // Agency Types
3,2021-03-24,4.22,UK-only // Growth Hacking
4,2021-03-25,44.98,US // Growth Hacking


In [10]:
from mmm.clean import unstack_data

df = unstack_data(df, metric_column='google_cost', unstack_column='campaign_name')

df.head()

,date,EU // Agency Types | google_cost,EU // Growth Hacking | google_cost,UK-only // Growth Hacking | google_cost,US // Agency Review Sites | google_cost,US // Agency Types | google_cost,US // Growth Hacking | google_cost
0,2021-03-24,9.82,4.08,4.22,0.47,106.99,72.66
1,2021-03-25,128.75,8.19,24.14,22.05,54.32,44.98
2,2021-03-26,100.24,0.00,0.00,22.65,30.01,0.00
3,2021-03-27,0.00,0.00,0.00,61.29,24.52,0.00
4,2021-03-28,0.00,0.00,0.00,17.85,1.64,0.00


In [11]:
df.to_csv('../data/google_clean.csv', index=False)

### Google Analytics
Load and clean Google Analytics data

In [12]:
# Load example Google Analytics data
file_name = "GoolyBib-trafficsources-all-traffic.csv"
data = pd.read_csv('../data/'+file_name)
data.head()

,# ----------------------------------------,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,# www.goolybib.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,# All Traffic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,# 20200426-20201022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,# ----------------------------------------,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.vexpower.com


In [15]:
data = pd.read_csv('../data/'+file_name, skiprows=6)
data.head()

,Date,Source/Medium,Revenue,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,4/26/2020,affiliate / cpa,14207.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/27/2020,affiliate / cpa,12761.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4/28/2020,affiliate / cpa,12474.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/29/2020,affiliate / cpa,12963.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/30/2020,affiliate / cpa,10441.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
from mmm.clean import rename_column

df = data.copy()
rename_column(df, 'Date', 'date')
rename_column(df, 'Source/Medium', 'source_medium')
rename_column(df, 'Revenue', 'revenue')

df = df[['date', 'revenue', 'source_medium']]
df.head()

,date,revenue,source_medium
0,4/26/2020,14207.75,affiliate / cpa
1,4/27/2020,12761.39,affiliate / cpa
2,4/28/2020,12474.80,affiliate / cpa
3,4/29/2020,12963.09,affiliate / cpa
4,4/30/2020,10441.81,affiliate / cpa


In [17]:
from mmm.clean import unstack_data

df = unstack_data(df, metric_column='revenue', unstack_column='source_medium')

df.head()

,date,(direct) / (none) | revenue,Total | revenue,affiliate / cpa | revenue,facebook / cpc | revenue,google / organic | revenue
0,10/1/2020,8261.61,52204.63,18442.77,7097.64,18402.62
1,10/10/2020,9407.63,46478.29,9342.80,5345.78,22382.09
2,10/11/2020,9151.77,46662.74,11730.68,11560.73,14219.56
3,10/12/2020,5147.25,35314.09,12423.72,8635.74,9107.38
4,10/13/2020,9063.16,48453.03,13360.96,11380.80,14648.11


In [ ]:
df.to_csv('../data/revenue_clean.csv', index=False)